# Practical LangGraph Examples

## Introduction

This notebook contains complete, production-ready examples of AI agents built with LangGraph:

1. **Research Assistant** - Gathers and synthesizes information
2. **Code Generator** - Writes and tests code
3. **Data Analyst** - Analyzes datasets and creates insights
4. **Customer Support** - Answers questions with knowledge base
5. **Task Planner** - Breaks down complex tasks

Each example is self-contained and ready to run!

## Example 1: Research Assistant Agent

An agent that can search for information, read sources, and synthesize findings.

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import operator

# 1. Define tools
@tool
def search_web(query: str) -> str:
    """Search the web for information."""
    # In production, use real search API (Tavily, SerpAPI)
    return f"""Search results for '{query}':
    1. LangGraph is a library for building stateful AI agents
    2. It extends LangChain with graph-based orchestration
    3. Supports cyclical flows and complex control logic
    """

@tool
def read_document(url: str) -> str:
    """Read and extract content from a URL."""
    # In production, use web scraping or PDF extraction
    return f"Content from {url}: Detailed information about the topic..."

# 2. Define state
class ResearchState(TypedDict):
    question: str
    messages: Annotated[List, operator.add]
    search_results: List[str]
    final_answer: str

# 3. Initialize LLM
llm = ChatOpenAI(model="gpt-4", temperature=0)
tools = [search_web, read_document]
llm_with_tools = llm.bind_tools(tools)

# 4. Define nodes
def research_node(state: ResearchState) -> ResearchState:
    """Agent conducts research"""
    messages = state.get("messages", [])
    if not messages:
        messages = [
            HumanMessage(content=f"""Research this question: {state['question']}
            Use search_web to find information. Be thorough.""")
        ]
    
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def synthesize_node(state: ResearchState) -> ResearchState:
    """Synthesize findings into final answer"""
    synthesis_prompt = f"""Based on the research conducted, provide a comprehensive answer to:
    {state['question']}
    
    Include:
    1. Key findings
    2. Supporting evidence
    3. Conclusion
    """
    
    messages = state["messages"] + [HumanMessage(content=synthesis_prompt)]
    response = llm.invoke(messages)
    
    return {"final_answer": response.content}

# 5. Build graph
research_workflow = StateGraph(ResearchState)

research_workflow.add_node("research", research_node)
research_workflow.add_node("synthesize", synthesize_node)

research_workflow.set_entry_point("research")
research_workflow.add_edge("research", "synthesize")
research_workflow.add_edge("synthesize", END)

research_agent = research_workflow.compile()

# 6. Test
print("Research Assistant Example")
print("=" * 50)

result = research_agent.invoke({
    "question": "What is LangGraph and how is it used?",
    "messages": [],
    "search_results": [],
    "final_answer": ""
})

print("\nFinal Answer:")
print(result["final_answer"])

## Example 2: Code Generator Agent

An agent that writes, tests, and debugs code.

In [ ]:
class CodeGenState(TypedDict):
    task: str
    code: str
    test_results: str
    iterations: int
    final_code: str

def generate_code(state: CodeGenState) -> CodeGenState:
    """Generate code based on task"""
    prompt = f"""Write Python code for: {state['task']}
    
    Requirements:
    - Include docstrings
    - Add type hints
    - Handle edge cases
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    code = response.content
    
    return {"code": code, "iterations": 0}

def test_code(state: CodeGenState) -> CodeGenState:
    """Test the generated code"""
    # In production, actually execute and test the code
    try:
        # Simulated testing
        exec(state["code"])  # Be careful with exec in production!
        test_results = "All tests passed!"
    except Exception as e:
        test_results = f"Error: {str(e)}"
    
    return {"test_results": test_results, "iterations": state["iterations"] + 1}

def debug_code(state: CodeGenState) -> CodeGenState:
    """Fix code issues"""
    prompt = f"""Fix this code:
    ```python
    {state['code']}
    ```
    
    Error: {state['test_results']}
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"code": response.content}

def should_debug(state: CodeGenState) -> str:
    """Decide if we need to debug"""
    if "passed" in state["test_results"].lower():
        return "finish"
    elif state["iterations"] >= 3:
        return "finish"  # Give up after 3 tries
    else:
        return "debug"

def finalize_code(state: CodeGenState) -> CodeGenState:
    """Finalize the code"""
    return {"final_code": state["code"]}

# Build graph
code_workflow = StateGraph(CodeGenState)

code_workflow.add_node("generate", generate_code)
code_workflow.add_node("test", test_code)
code_workflow.add_node("debug", debug_code)
code_workflow.add_node("finalize", finalize_code)

code_workflow.set_entry_point("generate")
code_workflow.add_edge("generate", "test")
code_workflow.add_conditional_edges(
    "test",
    should_debug,
    {"debug": "debug", "finish": "finalize"}
)
code_workflow.add_edge("debug", "test")
code_workflow.add_edge("finalize", END)

code_agent = code_workflow.compile()

# Test
print("Code Generator Example")
print("=" * 50)

result = code_agent.invoke({
    "task": "Write a function to check if a string is a palindrome",
    "code": "",
    "test_results": "",
    "iterations": 0,
    "final_code": ""
})

print("\nFinal Code:")
print(result["final_code"])

## Example 3: Customer Support Agent

An agent that answers customer questions using a knowledge base.

In [ ]:
class SupportState(TypedDict):
    customer_question: str
    category: str
    knowledge_base_results: List[str]
    response: str
    escalate: bool

# Simulated knowledge base
KNOWLEDGE_BASE = {
    "billing": "Billing information: Charges are processed monthly. Contact billing@example.com for issues.",
    "technical": "Technical support: Check our FAQ at example.com/faq or contact support@example.com",
    "account": "Account management: You can update your profile at example.com/profile"
}

def categorize_question(state: SupportState) -> SupportState:
    """Categorize the customer question"""
    prompt = f"""Categorize this customer question into one of:
    - billing
    - technical
    - account
    
    Question: {state['customer_question']}
    
    Respond with just the category name.
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    category = response.content.strip().lower()
    
    return {"category": category}

def search_knowledge_base(state: SupportState) -> SupportState:
    """Search knowledge base for relevant information"""
    results = KNOWLEDGE_BASE.get(state["category"], [])
    return {"knowledge_base_results": [results]}

def generate_response(state: SupportState) -> SupportState:
    """Generate customer response"""
    prompt = f"""Generate a helpful response to this customer:
    
    Question: {state['customer_question']}
    Category: {state['category']}
    Relevant Info: {state['knowledge_base_results']}
    
    Be friendly, professional, and helpful.
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"response": response.content, "escalate": False}

# Build graph
support_workflow = StateGraph(SupportState)

support_workflow.add_node("categorize", categorize_question)
support_workflow.add_node("search", search_knowledge_base)
support_workflow.add_node("respond", generate_response)

support_workflow.set_entry_point("categorize")
support_workflow.add_edge("categorize", "search")
support_workflow.add_edge("search", "respond")
support_workflow.add_edge("respond", END)

support_agent = support_workflow.compile()

# Test
print("Customer Support Example")
print("=" * 50)

result = support_agent.invoke({
    "customer_question": "Why was I charged twice this month?",
    "category": "",
    "knowledge_base_results": [],
    "response": "",
    "escalate": False
})

print(f"\nCategory: {result['category']}")
print(f"\nResponse: {result['response']}")

## Production Tips

### 1. Add Logging

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def logged_node(state):
    logger.info(f"Node executing with state: {state}")
    # Node logic
    result = process(state)
    logger.info(f"Node completed: {result}")
    return result

### 2. Add Monitoring

In [ ]:
import time

def monitored_node(state):
    start_time = time.time()
    
    try:
        result = process(state)
        duration = time.time() - start_time
        
        # Log metrics
        logger.info(f"Node completed in {duration:.2f}s")
        
        return result
    except Exception as e:
        logger.error(f"Node failed: {e}")
        raise

### 3. Add Rate Limiting

In [ ]:
from ratelimit import limits, sleep_and_retry

@sleep_and_retry
@limits(calls=10, period=60)  # 10 calls per minute
def call_llm(messages):
    return llm.invoke(messages)

## Key Takeaways

- **Modularity**: Break agents into reusable components
- **Error Handling**: Always handle failures gracefully
- **Testing**: Test each node independently
- **Monitoring**: Log and track agent behavior
- **Optimization**: Cache results, use appropriate models

## Next Steps

- Build your own agent for a specific use case
- Add persistence for long-running conversations
- Implement human-in-the-loop for critical decisions
- Deploy to production with proper monitoring

---

**Congratulations!** You've completed the LangGraph tutorials. Now go build amazing agents! 🚀